In [ ]:
# Importa as bibliotecas

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import matplotlib.pyplot as plt


In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

True

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

sqlalchemy.create_engine(cnx)


engine = create_engine(cnx)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from internacao;" 
df = pd.read_sql_query(query, cnx)

df

In [ ]:
# Inicializa as extensões do Panel necessárias para exibir tabelas 
# interativas (Tabulator) e notificações na interface gráfica

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)
pn.extension('matplotlib')

In [ ]:
def buscar_opcoes_pacientes():
    query = """
        SELECT nome, cpf FROM Paciente
    """
    df = pd.read_sql(query, engine)
    
    # usuario ver nome do paciente em vez de apenas seu cpf
    opcoes = {'--- Selecione ---': None}

    dados_banco = dict(zip(df['nome'] + " (" + df['cpf'] + ")", df['cpf']))
    opcoes.update(dados_banco)
    return opcoes

def buscar_opcoes_medicos():
    query = """
        SELECT u.nome as nome, m.crm as crm
        FROM medico m
        JOIN usuario u on u.login = m.login
    """
    df = pd.read_sql(query, engine)

    opcoes = {'--- Selecione ---': None}
    dados_banco = dict(zip(df['nome'] + " - CRM: " + df['crm'], df['crm']))
    opcoes.update(dados_banco)

    return opcoes
def buscar_opcoes_leitos():
    query = "SELECT numero FROM Leito" 
    
    df = pd.read_sql(query, engine)

    opcoes = df['numero'].tolist()
    opcoes.insert(0, 0)

    return opcoes

In [ ]:

flag=''

from datetime import datetime

select_leito = pn.widgets.Select(
    name='Selecione o Leito', 
    options=buscar_opcoes_leitos(),
    value=0
)

select_medico = pn.widgets.Select(
    name='Selecione o Médico Responsável', 
    options=buscar_opcoes_medicos(),
    value=None
)


select_paciente = pn.widgets.Select(
    name='Selecione o Paciente', 
    options=buscar_opcoes_pacientes(),
    value=None
)

data_hora_entrada = pn.widgets.DatetimePicker(
    name='Data e Hora da Entrada',
    value=datetime.now(),  
    enable_time=True, 
    disabled=False
)

data_hora_saida = pn.widgets.DatetimePicker(
    name='Data e Hora da Saida',
    value=datetime.now(),  
    enable_time=True, 
    disabled=False
)

id = pn.widgets.TextInput(
    name='Id',
    value='',
    disabled=False
)




In [ ]:
# Cria quatro botões para as ações principais da aplicação CRUD:
# Consultar, Inserir, Excluir e Atualizar registros no banco de dados


buttonConsultar = pn.widgets.Button(name='Consultar', button_type='default')

buttonInserir = pn.widgets.Button(name='Inserir', button_type='default')

buttonExcluir = pn.widgets.Button(name='Excluir', button_type='default')

buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='default')

In [ ]:


def queryAll():
    """
    Consulta todos os registros da tabela 'pessoa' no banco de dados e retorna
    um widget Tabulator para exibição interativa dos dados.

    Returns:
        pn.widgets.Tabulator: Widget que exibe a tabela com todos os dados da tabela 'pessoa'.
    """
    query = f"select * from internacao"
    df = pd.read_sql_query(query, cnx)
    return pn.widgets.Tabulator(df)


def on_consultar():
    """
    Consulta internações aplicando filtros.
    Se o campo estiver vazio, ele ignora aquele filtro.
    Se todos estiverem vazios, traz tudo.
    """
    cpf= select_paciente.value  
    crm= select_medico.value
    numero=select_leito.value
    try:
        
        sql = "SELECT * FROM Internacao WHERE 1=1"
        params = []
        
        
        if cpf: 
            sql += " AND cpf = %s"
            params.append(cpf)
            
        if crm:
            sql += " AND crm = %s"
            params.append(crm)
            
        if numero:
            sql += " AND numero = %s"
            params.append(int(numero)) 
        df = pd.read_sql(sql, engine, params=tuple(params))
        
        if df.empty:
            return pn.pane.Alert('Nenhum registro encontrado com esses filtros!', alert_type='warning')
        else:
            return pn.widgets.Tabulator(df, pagination='remote', page_size=10)

    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')


def on_inserir():
    cpf= select_paciente.value  
    crm= select_medico.value
    numero=select_leito.value
    """
    Insere um novo registro na tabela 'pessoa' usando os valores dos widgets
    nome, cpf, nascimento, sexo e salario.

    Returns:
        pn.widgets.Tabulator ou pn.pane.Alert: Tabela atualizada ou alerta em caso de erro.
    """

    try:
        
        
        check_leito = pd.read_sql(
            "SELECT id FROM Internacao WHERE numero = %s AND data_hora_saida IS NULL", 
            engine, 
            params=(numero,)
        )
        if not check_leito.empty:
            return pn.pane.Alert(f'⛔ ERRO: O Leito {numero} já está ocupado!', alert_type='danger')

        
        check_paciente = pd.read_sql(
            "SELECT numero FROM Internacao WHERE cpf = %s AND data_hora_saida IS NULL", 
            engine, 
            params=(cpf,)
        )
        if not check_paciente.empty:
            leito_ocupado = check_paciente.iloc[0]['numero']
            return pn.pane.Alert(f'⛔ ERRO: Este paciente já está internado no leito {leito_ocupado}!', alert_type='danger')

        if data_hora_saida.value is not None:
             if data_hora_saida.value < data_hora_entrada.value:
                 return pn.pane.Alert('⛔ ERRO: A data de saída não pode ser anterior à data de entrada!', alert_type='danger')

        cursor = con.cursor()
        query_insert = """
            INSERT INTO Internacao (numero, cpf, crm, data_hora_entrada, data_hora_saida) 
            VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(query_insert, (numero, cpf, crm, data_hora_entrada.value, data_hora_saida.value))
        con.commit()
        cursor.close()
        
        return queryAll()

    except Exception as e:
        if con:
            con.rollback()
        return pn.pane.Alert(f'Erro ao inserir: {str(e)}', alert_type='danger')

def on_atualizar():
    cpf= select_paciente.value  
    crm= select_medico.value
    numero=select_leito.value
    """
    Atualiza os campos nome e salario do registro identificado pelo CPF informado.

    Returns:
        pn.widgets.Tabulator ou pn.pane.Alert: Tabela atualizada ou alerta em caso de erro.
    """
    try:
        cursor= con.cursor()
        cursor.execute("UPDATE internacao SET numero = %s, crm = %s, data_hora_saida = %s, cpf = %s WHERE id = %s",
           (numero, crm, data_hora_saida.value ,cpf, id.value_input))
        cursor.query
        cursor.query
        con.commit()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}')


def on_excluir():

    """
    Exclui o registro da tabela 'pessoa' com o CPF informado.

    Returns:
        pn.widgets.Tabulator ou pn.pane.Alert: Tabela atualizada ou alerta em caso de erro.
    """
    try:
        cursor= con.cursor()
        cursor.execute("DELETE FROM internacao WHERE id=%s", (id.value_input,))        
        rows_deleted = cursor.rowcount
        con.commit()
        return queryAll()
    except:
        cursor.execute("ROLLBACK")            
        cursor.close() 
        return pn.pane.Alert('Não foi possível excluir!')


In [ ]:
# Função que chama a ação correta (consultar, inserir, atualizar ou excluir)
# dependendo do botão que foi clicado (representado pelos parâmetros booleanos)

def table_creator(cons, ins, atu, exc):
    if cons:
        return on_consultar()
    if ins:
        return on_inserir()
    if atu:
        return on_atualizar()
    if exc:
        return on_excluir()

In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

interactive_table = pn.bind(table_creator, buttonConsultar, buttonInserir, buttonAtualizar, buttonExcluir)

In [ ]:

def criar_graficos_pandas(event=None):

    # consulta sql que mostra as internacoes por departamento
    query_dep = """
        SELECT d.nome_dep, COUNT(*) as qtd 
        FROM Internacao i 
        JOIN Medico m ON i.crm = m.crm 
        JOIN Departamento d ON m.nome_dep = d.nome_dep 
        GROUP BY d.nome_dep
    """
    df_dep = pd.read_sql(query_dep, engine)
    
    fig1, ax1 = plt.subplots(figsize=(6, 5))
    
    if not df_dep.empty:
        df_dep.set_index('nome_dep').plot(
            kind='pie',           
            y='qtd',              
            ax=ax1,               
            autopct='%1.1f%%',   
            legend=False,         
            title='Internações por Departamento',
            ylabel=''             
        )
    else:
        ax1.text(0.5, 0.5, "Sem dados", ha='center')

    # consulta para tipo de leito ocupado na internacao
    query_leito = """
        SELECT l.tipo, COUNT(*) as qtd 
        FROM Internacao i 
        JOIN Leito l ON i.numero = l.numero 
        GROUP BY l.tipo
    """
    df_leito = pd.read_sql(query_leito, engine)
    
    fig2, ax2 = plt.subplots(figsize=(8, 4))
    
    if not df_leito.empty:
        df_leito.set_index('tipo').plot(
            kind='bar', 
            y='qtd', 
            ax=ax2, 
            color='#4CAF50',      
            rot=0,                 
            legend=False,
            title='Ocupação por Tipo de Leito'
        )
        ax2.set_xlabel('Tipo de Leito')
    else:
        ax2.text(0.5, 0.5, "Sem dados", ha='center')

    plt.close(fig1)
    plt.close(fig2)

    return pn.Column(
        pn.pane.Matplotlib(fig1, tight=True),
        pn.pane.Matplotlib(fig2, tight=True),
        name="Gráficos"
    )

layout_crud = pn.Row(
    pn.Column(
        '## 🏥 Gestão de Internações',
        select_leito, select_paciente, select_medico, 
        data_hora_entrada, data_hora_saida, id,
        pn.Row(buttonConsultar, buttonInserir),
        pn.Row(buttonAtualizar, buttonExcluir)
    ),
    pn.Column(interactive_table),
    name="Gerenciamento"
)

# o grafico recarrega sempre que insere dados
layout_dashboard = pn.bind(criar_graficos_pandas, buttonInserir)

aba_principal = pn.Tabs(
    layout_crud,
    ("📊 Dashboard (Pandas)", layout_dashboard)
)

aba_principal.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] TextInput(name='Nome', placeholder='Digite o nome')
        [2] TextInput(name='CPF ', placeholder='Digite o CPF')
        [3] DatePicker(name='Data de Nascimento')
        [4] RadioBoxGroup(name='Sexo', options=['Não Informado', ...], value='Não Informado')
        [5] FloatInput(end=10000000, name='Salario', start=0, value=1000.0)
        [6] Row
            [0] Button(name='Consultar')
        [7] Row
            [0] Button(name='Inserir')
        [8] Row
            [0] Button(name='Atualizar')
        [9] Row
            [0] Button(name='Excluir')
    [1] Column
        [0] ParamFunction(function, _pane=Str, defer_load=False)